# **Taller 2**
Este proyecto tiene como objetivo explorar diversas estrategias de ingeniería de características en el contexto de un problema de regresión lineal utilizando el conjunto de datos de precios de casas de Boston. Se utilizará un modelo de regresión lineal estándar y se evaluará su desempeño en diferentes escenarios de preprocesamiento de datos.

In [10]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import KBinsDiscretizer

# **1. Carga del Dataset**
En esta sección, se carga el conjunto de datos de Boston utilizando una URL específica y se organiza en un formato adecuado. Se divide en características (X) y el objetivo (y).

In [9]:
# Cargar el dataset de Boston
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
raw_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.00632,18.00,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3
1,396.90000,4.98,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02731,0.00,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8
3,396.90000,9.14,21.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02729,0.00,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8


# **2. División del Conjunto de Datos**
Se divide el conjunto de datos en conjuntos de entrenamiento y prueba.

In [11]:
# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# **3. Entrenamiento y Evaluación con Características Originales**
Se entrena un modelo de regresión lineal estándar con las características originales y se evalúa su rendimiento.

In [24]:
# Inicializar el modelo de regresión lineal estándar
linear_model = LinearRegression()
# Entrenar el modelo con características originales
linear_model.fit(X_train, y_train)
y_pred_original = linear_model.predict(X_test)
# Evaluar el modelo con características originales
mse_original = mean_squared_error(y_test, y_pred_original)
r2_original = r2_score(y_test, y_pred_original)

# **4. Entrenamiento y Evaluación con Características Escaladas**
Se escalan las características utilizando tres métodos diferentes (StandardScaler, MinMaxScaler, RobustScaler) y se evalúa el rendimiento del modelo para cada caso.

In [37]:
# Escalar características utilizando diferentes métodos
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
scaled_results = {}

for scaler in scalers:
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Entrenar el modelo con características escaladas
    linear_model.fit(X_train_scaled, y_train)
    y_pred_scaled = linear_model.predict(X_test_scaled)

    # Evaluar el modelo con características escaladas
    mse_scaled = mean_squared_error(y_test, y_pred_scaled)
    r2_scaled = r2_score(y_test, y_pred_scaled)

    scaled_results[type(scaler).__name__] = {'MSE': mse_scaled, 'R2': r2_scaled}

# **5. Entrenamiento y Evaluación con Características Proyectadas por PCA**
Se utiliza PCA para proyectar las características originales en un espacio de dos dimensiones y se evalúa el rendimiento del modelo.

In [16]:
# Proyectar características sobre los dos primeros componentes principales (PCA)
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Entrenar el modelo con características proyectadas por PCA
linear_model.fit(X_train_pca, y_train)
y_pred_pca = linear_model.predict(X_test_pca)

# Evaluar el modelo con características proyectadas por PCA
mse_pca = mean_squared_error(y_test, y_pred_pca)
r2_pca = r2_score(y_test, y_pred_pca)

# **6. Entrenamiento y Evaluación con Características Después de Binning**
Se utiliza KBinsDiscretizer para discretizar las características y se evalúa el rendimiento del modelo.

In [ ]:
# Usar binning y características de interacción (solo para características continuas)
n_bins_values = range(2,13)
mse_values_binned = []
r2_values_binned = []
for n in n_bins_values:
  kbins_discretizer = KBinsDiscretizer(n_bins=n, encode='ordinal', strategy='quantile')
  X_train_binned = kbins_discretizer.fit_transform(X_train)
  X_test_binned = kbins_discretizer.transform(X_test)

  # Entrenar el modelo con características después de binning
  linear_model.fit(X_train_binned, y_train)
  y_pred_binned = linear_model.predict(X_test_binned)

  # Evaluar el modelo con características después de binning
  mse_binned = mean_squared_error(y_test, y_pred_binned)
  r2_binned = r2_score(y_test, y_pred_binned)

  mse_values_binned.append(mse_binned)
  r2_values_binned.append(r2_binned)

# **7. Entrenamiento y Evaluación con Características Polinómicas**
Se utiliza PolynomialFeatures para generar características polinómicas de diferentes grados y se evalúa el rendimiento del modelo para cada caso. (Originalmente se deberia hacer el analisis entre 2 y 12 pero la ram no es suficiente para hacerlo con grados muy altos)

In [22]:
# Usar características polinómicas de diferentes grados
degrees = range(2, 7)
poly_results = {}

for degree in degrees:
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Entrenar el modelo con características polinómicas
    linear_model.fit(X_train_poly, y_train)
    y_pred_poly = linear_model.predict(X_test_poly)

    # Evaluar el modelo con características polinómicas
    mse_poly = mean_squared_error(y_test, y_pred_poly)
    r2_poly = r2_score(y_test, y_pred_poly)

    poly_results[degree] = {'MSE': mse_poly, 'R2': r2_poly}

# **8. Impresión de Resultados**
Se imprimen los resultados de todas las estrategias.

In [42]:
# Imprimir resultados
print("Resultados con características originales:")
print(f"MSE: {mse_original:.4f}, R2: {r2_original:.4f}\n")

for scaler_name, result in scaled_results.items():
    print(f"Resultados con características escaladas ({scaler_name}):")
    print(f"MSE: {result['MSE']:.4f}, R2: {result['R2']:.4f}")
print()

print("Resultados con características proyectadas por PCA:")
print(f"MSE: {mse_pca:.4f}, R2: {r2_pca:.4f}\n")

print("Resultados con binning:")
for i, n_bins in enumerate(n_bins_values):
    print(f"Grado {n_bins}: MSE: {mse_values_binned[i]:.4f}, R2: {r2_values_binned[i]:.4f}")
print()

print("Resultados con características polinómicas:")
for degree, results in poly_results.items():
    print(f"Grado {degree}: MSE: {results['MSE']:.4f}, R2: {results['R2']:.4f}")

Resultados con características originales:
MSE: 24.2911, R2: 0.6688

Resultados con características escaladas (StandardScaler):
MSE: 24.2911, R2: 0.6688
Resultados con características escaladas (MinMaxScaler):
MSE: 24.2911, R2: 0.6688
Resultados con características escaladas (RobustScaler):
MSE: 24.2911, R2: 0.6688

Resultados con características proyectadas por PCA:
MSE: 55.0173, R2: 0.2498

Resultados con binning:
Grado 2: MSE: 32.5295, R2: 0.5564
Grado 3: MSE: 29.4086, R2: 0.5990
Grado 4: MSE: 24.3837, R2: 0.6675
Grado 5: MSE: 27.7237, R2: 0.6220
Grado 6: MSE: 23.3972, R2: 0.6809
Grado 7: MSE: 23.6622, R2: 0.6773
Grado 8: MSE: 24.1250, R2: 0.6710
Grado 9: MSE: 23.0292, R2: 0.6860
Grado 10: MSE: 23.2744, R2: 0.6826
Grado 11: MSE: 22.0658, R2: 0.6991
Grado 12: MSE: 21.0513, R2: 0.7129

Resultados con características polinómicas:
Grado 2: MSE: 14.5664, R2: 0.8014
Grado 3: MSE: 129848.0690, R2: -1769.6446
Grado 4: MSE: 5417.5370, R2: -72.8751
Grado 5: MSE: 5137.5724, R2: -69.0574
Grado 

# **Conclusiones**

#### **Características Originales**
El modelo entrenado con las características originales presenta un MSE de 24.2911 y un R2 de 0.6688.

#### **Características Escaladas**
Al observar los resultados con características escaladas utilizando diferentes métodos (StandardScaler, MinMaxScaler, RobustScaler), no se evidencia una mejora significativa en el rendimiento del modelo. Los resultados son consistentes entre los diferentes escaladores, manteniendo un MSE de 24.2911 y un R2 de 0.6688. En este caso, el escalado no aporta mejoras sustanciales al modelo.

#### **Características Proyectadas por PCA**
La proyección de características en dos dimensiones utilizando PCA resulta en una reducción del rendimiento del modelo. El MSE aumenta a 55.0173, y el R2 disminuye a 0.2498. Esta estrategia de reducción de dimensionalidad no es beneficiosa para el conjunto de datos.

#### **Características Después de Binning**
La discretización mediante binning ofrece resultados variables según el grado. Se observa que el binning con grado 2 proporciona un MSE de 32.5295 y un R2 de 0.5564, y a medida que el grado aumenta, el rendimiento varía. En general, el binning no logra superar el rendimiento del modelo con características originales.

#### **Características Polinómicas**
El uso de características polinómicas de grado 2 resulta en el mejor rendimiento, con un MSE de 14.5664 y un R2 de 0.8014. Sin embargo, a medida que el grado polinómico aumenta, el rendimiento empeora significativamente, con MSE y R2 fuera de rango práctico en grados superiores.

## **Estrategia de Ingeniería de Características**
La estrategia de ingeniería de características que ofrece los mejores resultados es la utilización de características polinómicas de grado 2. Este enfoque logra un equilibrio entre la complejidad del modelo y la capacidad para capturar la relación no lineal en los datos, como se evidencia por el bajo MSE y el alto R2. Los demás métodos (escalado, PCA, binning) no ofrecen mejoras sustanciales y, en algunos casos, empeoran el rendimiento del modelo. La elección de esta estrategia se basa en la capacidad del modelo para capturar la complejidad no lineal de los datos, sin incurrir en la alta complejidad asociada a grados polinómicos mayores.
